<h1> Run to Mount Drive Instance with Current Data (Must Run First) <h1>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


<h1> Format (.txt) Files to (.csv) Data <h1>


The code below takes the folder of stock indices and converts the entire folder of txt files to a single csv.

In [ ]:
import os
import pandas as pd

# Define the folder path containing the .txt files and the output CSV file.
FOLDER_PATH = "/content/drive/MyDrive/CSC442 Team Project/Datasets/world/stooq stocks indices"  # Change this to your folder location
OUTPUT_FILE = "/content/drive/MyDrive/CSC442 Team Project/Output/StockData.csv"  # Change the output file name if needed

def merge_txt_to_csv(folder_path, output_file):
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    merged_data = []

    if not all_files:
        print("No text files found in the folder.")
        return

    for file in all_files:
        file_path = os.path.join(folder_path, file)
        try:
            # Read the file into a DataFrame
            df = pd.read_csv(file_path, delimiter=',', header=0)  # Adjust delimiter if needed
            df['Source'] = file  # Add a column to track source file name
            merged_data.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    if merged_data:
        stock_df = pd.concat(merged_data, ignore_index=True)
        stock_df.to_csv(output_file, index=False)
        print(f"Merged CSV saved as: {output_file}")
        return stock_df
    else:
        print("No valid data to merge.")

# Run the function
stock_df = merge_txt_to_csv(FOLDER_PATH, OUTPUT_FILE)

Merged CSV saved as: /content/drive/MyDrive/CSC442 Team Project/Output/mergedStockData.csv


<h1> Selects Valid Countries from GEI Data <h1>

<h2> Countries: Germany, China(HK), Hungary, Japan, Poland, UK, US <h2

In [ ]:
import pandas as pd


# 1. Read in the econ data

df = pd.read_csv("/content/drive/MyDrive/CSC442 Team Project/Datasets/Global Economy Indicators.csv")

# Strip whitespace from column names
df.columns = df.columns.str.strip()
# Strip whitespace from Country names
df["Country"] = df["Country"].str.strip()
# Print column names to verify
print(df.columns)

# 2. Define the list of valid countries
valid_countries = [
    "Germany",                  # ^de
    "China, Hong Kong SAR",     # ^hk
    "Hungary",                  # ^hu
    "Japan",                    # ^jp
    "Poland",                   # ^pl, ^pl20, ^plnc, ^plws
    "United Kingdom",           # ^uk
    "United States"             # ^us, ^usnm, ^usnq, ^usns
]

# 3. Filter the DataFrame so only rows with these countries remain
gei_df = df[df["Country"].isin(valid_countries)]

# 4. Write filtered data to a new CSV
gei_df.to_csv("/content/drive/MyDrive/CSC442 Team Project/Output/econDataFilteredCountries.csv", index=False)

print("Filtered CSV saved as econDataFilteredCountries.csv")

Index(['CountryID', 'Country', 'Year', 'AMA exchange rate',
       'IMF based exchange rate', 'Population', 'Currency', 'Per capita GNI',
       'Agriculture, hunting, forestry, fishing (ISIC A-B)',
       'Changes in inventories', 'Construction (ISIC F)',
       'Exports of goods and services', 'Final consumption expenditure',
       'General government final consumption expenditure',
       'Gross capital formation',
       'Gross fixed capital formation (including Acquisitions less disposals of valuables)',
       'Household consumption expenditure (including Non-profit institutions serving households)',
       'Imports of goods and services', 'Manufacturing (ISIC D)',
       'Mining, Manufacturing, Utilities (ISIC C-E)',
       'Other Activities (ISIC J-P)', 'Total Value Added',
       'Transport, storage and communication (ISIC I)',
       'Wholesale, retail trade, restaurants and hotels (ISIC G-H)',
       'Gross National Income(GNI) in USD', 'Gross Domestic Product (GDP)'],
    

<h1> Data Merging <h1>

In [ ]:
import datetime

mut_stock_df = stock_df

# Remove unecessary columns from GEI data.
selected_columns = ['Country', 'Year', 'Gross Domestic Product (GDP)', 'Gross National Income(GNI) in USD', 'Population', '']
gei_df = gei_df.loc[:, selected_columns]
gei_df = gei_df[gei_df['Year'] >= 1991]

#Remove unecessary columns from stock data.
selected_columns = ['<DATE>', '<TICKER>', '<OPEN>', '<CLOSE>', '<VOL>']
mut_stock_df = mut_stock_df.loc[:, selected_columns]
mut_stock_df['<TICKER>'] = mut_stock_df['<TICKER>'].str.strip("^_")

# Merge datasets by allocating GEI data to stock data by date & country.
mut_stock_df['Year'] = mut_stock_df['<DATE>'] // 10000
mut_stock_df = mut_stock_df[mut_stock_df['Year'] <= 2021]
mut_stock_df['<DATE>'] = pd.to_datetime(mut_stock_df['<DATE>'], format='%Y%m%d', errors='coerce').dt.strftime('%m-%d')

mapping_df = pd.DataFrame({'Country': ['Germany','China, Hong Kong SAR', 'Hungary', 'Japan', 'Poland', 'Poland', 'Poland', 'Poland', 'United Kingdom', 'United States', 'United States', 'United States', 'United States'],
                           '<TICKER>': ['DE', 'HK', 'HU', 'JP', 'PL', 'PL20', 'PLNC', 'PLWS', 'UK', 'US', 'USNM', 'USNQ', 'USNS']})

mut_stock_df = mut_stock_df.merge(mapping_df, on='<TICKER>', how='left')
merged_stock_df = mut_stock_df.merge(gei_df, on=['Country', 'Year'], how='left')

merged_stock_df.rename(columns={'<DATE>': 'Date',
                          'Gross Domestic Product (GDP)': 'GDP',
                          'Gross National Income(GNI) in USD': 'GNI (USD)',
                          'AMA exchange rate': 'Exchange Rate (AMA)'},  inplace=True)

mut_stock_df['Date'] = yearly_avg['Date'].dt.strftime('%Y%m%d')

# Write final dataset to output as csv.
merged_stock_df.to_csv("/content/drive/MyDrive/CSC442 Team Project/Output/final5b.csv", index=False)

mut_stock_df.head()

,Date,<TICKER>,<OPEN>,<CLOSE>,<VOL>,Year,Country,GDP,GNI (USD),Population,Exchange Rate (AMA)
0,02-16,USNM,833.46,835.37,6.963462e+07,2016,United States,1.870000e+13,1.900000e+13,327210198,1.000000
1,02-17,USNM,838.96,848.24,6.826358e+07,2016,United States,1.870000e+13,1.900000e+13,327210198,1.000000
2,02-18,USNM,848.99,848.14,6.411843e+07,2016,United States,1.870000e+13,1.900000e+13,327210198,1.000000
3,02-19,USNM,849.00,845.80,6.539245e+07,2016,United States,1.870000e+13,1.900000e+13,327210198,1.000000
4,02-22,USNM,851.78,853.42,6.515698e+07,2016,United States,1.870000e+13,1.900000e+13,327210198,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
24386,12-23,UK,950.01,955.47,2.900083e+09,2021,United Kingdom,3.130000e+12,3.120000e+12,67281039,0.727065
24387,12-24,UK,955.54,958.02,1.437548e+09,2021,United Kingdom,3.130000e+12,3.120000e+12,67281039,0.727065
24388,12-29,UK,953.36,960.49,3.477622e+09,2021,United Kingdom,3.130000e+12,3.120000e+12,67281039,0.727065
24389,12-30,UK,959.52,962.81,3.140106e+09,2021,United Kingdom,3.130000e+12,3.120000e+12,67281039,0.727065


# Overall Data Merge plan:

Initial filtering: remove outliers/missing values before doing any of this?

For economic indicators (data on many countries):
- remove all rows on countries that don't match those in the stock data
- remove columns for sub-industry data (only really need GDP, GNI, population, country, year; maybe exchange rates?)

For stock market (data on a select few countries):
- filter daily data into average per day
- filter the simplified daily data into descriptive stats by year (mean, median, maybe range?)
- ensure there is a standard unit (USD)


Final merge:
- By country by year
- Remove rows with missing values/outliers
- Country and currency work as categorical variables: can answer individual questions by these as well